### Pydantic
pydantic models provide the richest feature set with field validation, descriptions and nested structures

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model =init_chat_model("groq:qwen/qwen3-32b")

In [2]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000182210A9590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000182212196D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")



In [4]:
model_with_structured_output = model.with_structured_output(Movie)
model_with_structured_output

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000182210A9590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000182212196D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', 'dire

In [5]:
model_with_structured_output.invoke("Provide details of Movie Bahubali2")

Movie(title='Bahubali2', year=2017, director='S.S. Rajamouli', rating=8.3)

In [6]:
model_with_structured_output.invoke("Provide details of Movie RRR")

Movie(title='RRR', year=2022, director='S. S. Rajamouli', rating=8.8)

In [8]:
class Movie(BaseModel):
    """A Movie with details"""
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

model_with_structure = model.with_structured_output(Movie,include_raw=True)

response = model_with_structure.invoke("Provide Details about Movie Bahubali")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Bahubali. I need to use the Movie function provided. Let me check the required parameters: title, year, director, and rating. I know the title is "Bahubali". The release year was 2015. The director is S.S. Rajamouli. As for the rating, I think it\'s around 8.0 on IMDb. Let me confirm those details. Yeah, that\'s correct. So I\'ll structure the tool call with those parameters.\n', 'tool_calls': [{'id': '0mrdqydn4', 'function': {'arguments': '{"director":"S.S. Rajamouli","rating":8,"title":"Bahubali","year":2015}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 229, 'total_tokens': 394, 'completion_time': 0.287758861, 'completion_tokens_details': {'reasoning_tokens': 111}, 'prompt_time': 0.011775351, 'prompt_tokens_details': None, 'queue_time': 0.057984759, 'total_time': 0.299534212}, 'model_nam

### Nested Structure

In [9]:
class Actor(BaseModel):
    name:str
    role:str


class MovieDetails(BaseModel):
    title:str
    year:str
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")


model_with_structure  = model.with_structured_output(MovieDetails)
response = model_with_structure.invoke("Provide Details of Movie Bahubali2")
response

MovieDetails(title='Baahubali 2: The Conclusion', year='2017', cast=[Actor(name='Prabhas', role='Baahubali'), Actor(name='Anushka Shetty', role='Devasena'), Actor(name='Rana Daggubati', role='Bhallaladeva')], genres=['Action', 'Adventure', 'Fantasy'], budget=100.0)